In [8]:
import importlib
import classification_training_utils
importlib.reload(classification_training_utils)
import utils
importlib.reload(utils)

import sys
sys.path.append('../')

import pandas as pd
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util
from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import TripletEvaluator
from sklearn.metrics.pairwise import cosine_similarity
from utils import load_model, replace_nan_with, load_big_consulting_export, callback, create_replace_no_tags_embeddings
import numpy as np
import random
from torch.utils.data import DataLoader
from datetime import datetime
import os
from collections import defaultdict
import S3BERT.src.model_freeze as freeze
import matplotlib.pyplot as plt
plt.style.use('ggplot')


from classification_training_utils import get_big_consulting_df, collect_classification_labels, get_relevant_classifications, train, filter_relevant_classifications, get_top_values, get_news_df
from transformers import EarlyStoppingCallback
import math
import torch
from tqdm import tqdm
tqdm.pandas()
import pickle

In [9]:
params = {}
params["EPOCHS"] = 4
params["USE_REPLACE_DATA"] = True
params["USE_ORIGINAL_DATA"] = False
params["UNFREEZE_LAYERS"] = 2
params["EXCLUDE_ENTITY_OTHER"] = True
params["INITIALIZED_MODEL"] = None #"brjezierski/S3BERT" # "intfloat/e5-small-v2"
params["OCCURENCE_CUTOFF"] = 2
params["CREATE_NEW_SPLIT"] = False
params["BATCH_SIZE"] = 32

dataset_dir = "../classification-training-data/" #5-9-23/"
sbert_model = load_model() if not params["INITIALIZED_MODEL"] else load_model(
        model=params["INITIALIZED_MODEL"])

In [10]:


prefix = "../glanos-data/embeddings/"

with open(f'{prefix}big_consulting_2_snippet.pickle', 'rb') as f:
# with open(f'{prefix}big_consulting_2_replace_no_tags.pickle', 'rb') as f:
    replace_no_tags_embeddings = pickle.load(f)

replace_data_path = '../glanos-data/datasets/big_consulting_export_replace.tsv'

if os.path.exists(replace_data_path):
    df = pd.read_csv(replace_data_path, sep='\t')
    df['embedding'] = df['snippet'].map(replace_no_tags_embeddings)
#     df = create_replace_no_tags_embeddings(df, replace_no_tags_embeddings)
#     df['embedding'] = df['replace_no_tags'].map(replace_no_tags_embeddings)

df = collect_classification_labels(df, verbose=True)
df = get_top_values(df, params)
all_classifications, relevant_classifications = get_relevant_classifications(df, params)
df, classifications = filter_relevant_classifications(df, all_classifications, relevant_classifications)


['LEADERSHIP', 'STAFFING-LAYOFFS', '^SELL-TO', 'STATE', 'SUPPLY', 'ACQUIRED', 'OP_LOC', '^EMPLOY', 'FIGHT-ABUSE-HUM', 'COWORK', 'DEAL', 'COMPETENCE', 'GROW-MARKET', 'PENALTY-IP', 'SELL', 'USE-PROD', 'MARKET_LOC', 'MEDIASCORE', '^USE', 'AGREEMENT', 'SUE-END', '^SELL', 'PRODUCT-ADV', 'PRODUCT', 'CAREER', 'REGULATORY', 'TECH-PATENT', 'RIGHTS', 'MARKET-COVERAGE', 'CLOSE-FACILITY', 'GROW-FORECAST', 'PROBLEM', 'DECLINE-MARKET', 'EMPLOY-STOP', 'FUNDING', 'GROW-PROD', 'FOUNDER', 'GROW-HIRE', 'PRODUCT-STOP', 'OPERATE-LAUNCH', '^SUPPLY', 'ACQUIRE', 'MARKET', 'DECLINE', 'EMPLOY', 'PARTNER_JV', 'SERVICE', 'EVENT', 'TECH', '^DEAL', 'PARTNER', 'STAFFING', 'EMPLOY-START', 'GENERAL', 'STRATEGY', 'ENTITY', 'GROW-FACILITY', 'ACTIVITY_LOC', 'ABUSE-LAW', 'ABUSE-BR', 'PENALTY', 'AWARD-SPONS', 'EFFICIENCY', 'INVEST', 'COMPETE', 'GROW', 'OTHER', '^SERVICE', 'PR', 'CRITIC', 'PROBLEM-SOLUTION', 'CHARITY', 'CORP_REPORT', 'LAUNCH', 'DECISION', 'GROW-NEUTRAL', 'AWARD', 'SUE-WIN', 'ACQUIRE-ASSET', 'INTERNAL', 'USE

In [11]:
params["WARMUP_STEPS"] = int(len(df) * params["EPOCHS"] * 0.1)  # 10% of train data
# df['snippet'] = df['replace_no_tags']

In [12]:
model_fit, test_evaluator, sbert_model = train(df, classifications, params, dataset_dir, sbert_model)


Training with one objective
e=4 Using replacement data, unfreezing 2 last layers, excluding Entity and Other labels, only including words that occur at least 2 times, training data size2585

Training data size 2585
Validation data size 100
Test data size 100
Performance before fine-tuning:


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

Score at epoch 0, step -1: 0.78


Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

Score at epoch 1, step -1: 0.8


Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

Score at epoch 2, step -1: 0.82


Iteration:   0%|          | 0/80 [00:00<?, ?it/s]

Score at epoch 3, step -1: 0.84


In [13]:
sbert_model.evaluate(test_evaluator)

0.84

In [14]:
frozen_model = load_model()
frozen_model.evaluate(test_evaluator)

0.77